In [ ]:
!pip install pykeops[colab] > log.log

In [ ]:
import pykeops
pykeops.clean_pykeops()          # just in case old build files are still present

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch

import time

from nystrom_numpy import Nystrom as Nystrom_NK
from nystrom_torch import Nystrom as Nystrom_TK


test_data = np.random.randint(1,10,(100,4)).astype(np.float32)
test_data_t = torch.tensor(test_data, dtype=torch.float32)

In [ ]:
test_data64 = np.random.randint(1,10,(100,4)).astype(np.float64)
test_data_t64 = torch.tensor(test_data, dtype=torch.float64)

###Unit tests: Numpy

In [ ]:
class UnitTestCaseNumpy():


    # add these to the init part in keops
    def __init__(self, data, dtype):
        self.length = 100
        self.num_sampling = 40
        self.x = data
        self.dtype = dtype

        # self.k_rbf = None
        # self.k_exp = None

    ############################################################    
    def Nystrom_K_approx_test(self):
        ############################################################
        
        # from pykeops.numpy.nystrom import Nystrom_NK
        ''' 
        Function to test K_approx method.
        We expect K_approx(x) ~ X_new @ X_new^T.
    
        '''
        kernels = ['rbf', 'exp']
        
        for kernel in kernels:

            N_truth = Nystrom_NK(n_components=self.length, kernel = kernel, random_state=0).fit(self.x)
            x_truth = N_truth.transform(self.x, dense=True)
            K = x_truth @ x_truth.T
            # if kernel == 'rbf':
            #     self.k_rbf = K
            # else:
            #     self.k_exp = K
            # print(f"{kernel} true kernel \n", K, "\n")

            N_NK = Nystrom_NK(n_components=self.num_sampling, kernel = kernel, random_state=0).fit(self.x)
            K_approx_1 = N_NK.K_approx(self.x)
            K_approx_1 = K_approx_1 @ np.identity(K_approx_1.shape[0], dtype=self.dtype)

            x_new = N_NK.transform(self.x, dense=True)
            K_approx_2 = x_new @ x_new.T

            # print(f"{kernel} kernel K_approx 1 \n", K_approx_1, "\n")
            # print(f"{kernel} kernel K_approx 2 \n", K_approx_2, "\n")

            approx_1_e = np.linalg.norm(K-K_approx_1) / K.size
            approx_2_e = np.linalg.norm(K-K_approx_2) / K.size
            print(f"{kernel} kernel K_approx 1 error", np.linalg.norm(K-K_approx_1) / K.size )
            print(f"{kernel} kernel K_approx 2 error", np.linalg.norm(K-K_approx_2) / K.size )
            ML2_error = np.linalg.norm(K_approx_1 - K_approx_2) / K.size
            print(f"ML2 error for {kernel} kernel:", ML2_error, "\n")
            # print("-"*100, "\n")

            assert approx_1_e < 0.1
            assert approx_2_e < 0.1
            assert ML2_error < 0.01
            #self.assertTrue(ML2_error < 0.01)

    ############################################################ 
    def Nystrom_K_shape_test(self):
        ############################################################

        # from pykeops.numpy.nystrom import Nystrom_NK 
        ''' 
        Function to check that data shapes are correct.
    
        '''
        kernels = ['rbf', 'exp']
        
        for kernel in kernels:
            N_NK = Nystrom_NK(n_components=self.num_sampling, kernel = kernel, random_state=0).fit(self.x)

            assert N_NK.normalization_.shape == (self.num_sampling, self.num_sampling)
            assert N_NK.transform(self.x).shape == (self.length, self.num_sampling)

            #self.assertTrue(N_NK.normalization_.shape == (self.num_sampling, self.num_sampling))
            #self.assertTrue(N_NK.transform(self.x).shape == (self.length, self.num_sampling))

print("float32 data")     
test_np = UnitTestCaseNumpy(test_data, np.float32)    
test_np.Nystrom_K_approx_test()
test_np.Nystrom_K_shape_test()

print("\n", "float64 data") 
test_np = UnitTestCaseNumpy(test_data64, np.float64)    
test_np.Nystrom_K_approx_test()
test_np.Nystrom_K_shape_test()

float32 data
rbf kernel K_approx 1 error 0.0007300935745239257
rbf kernel K_approx 2 error 0.0007300935745239257
ML2 error for rbf kernel: 1.541538608762494e-10 

exp kernel K_approx 1 error 0.0006973778247833252
exp kernel K_approx 2 error 0.0006973778247833252
ML2 error for exp kernel: 2.3594138838234357e-10 


 float64 data
rbf kernel K_approx 1 error 0.0007384496675632467
rbf kernel K_approx 2 error 0.0007384496675632467
ML2 error for rbf kernel: 2.744249517285973e-19 

exp kernel K_approx 1 error 0.0006970258422272258
exp kernel K_approx 2 error 0.0006970258422272258
ML2 error for exp kernel: 3.9925512931787905e-19 



###Unit tests: Torch

In [ ]:
class UnitTestCase():
    
    def __init__(self, data, dtype):

        # add these to the init part in keops
        self.length = 100
        self.num_sampling = 40
        self.x = data
        self.dtype = dtype

        # self.k_rbf = None
        # self.k_exp = None

    ############################################################         
    def Nystrom_K_approx_test(self):
        ############################################################ 

        # from pykeops.torch.nystrom import LazyNystrom_TK as Nystrom_TK
        ''' 
        Function to test K_approx method.
        We expect K_approx(x) ~ X_new @ X_new^T.
    
        '''
        kernels = ['rbf', 'exp']
        
        for kernel in kernels:

            N_truth = Nystrom_TK(n_components=self.length, kernel = kernel, random_state=0).fit(self.x)
            x_truth = N_truth.transform(self.x, dense=True)
            K = x_truth @ x_truth.T

            # if kernel == 'rbf':
            #     self.k_rbf = K
            # else:
            #     self.k_exp = K
            # print(f"{kernel} true kernel \n", K, "\n")

            N_TK = Nystrom_TK(n_components=self.num_sampling, kernel = kernel, random_state=0).fit(self.x)
            K_approx_ = N_TK.K_approx(self.x)
            id = torch.tensor(torch.diag(torch.ones(self.length)), dtype=self.dtype)
            K_approx_1 = K_approx_ @ id

            x_new = N_TK.transform(self.x, dense=True)
            K_approx_2 = x_new @ x_new.T

            # print(f"{kernel} kernel K_approx 1 \n", K_approx_1, "\n")
            # print(f"{kernel} kernel K_approx 2 \n", K_approx_2, "\n")
            print(f"{kernel} kernel K_approx 1 error", (torch.linalg.norm(K-K_approx_1)/ (K.shape[0]*K.shape[1])).item())
            print(f"{kernel} kernel K_approx 2 error", (torch.linalg.norm(K-K_approx_2) / (K.shape[0]*K.shape[1])).item() )
            
            approx_1_e = torch.linalg.norm(K-K_approx_1) / (K.shape[0]*K.shape[1])
            approx_2_e = torch.linalg.norm(K-K_approx_2) / (K.shape[0]*K.shape[1])

            ML2_error = torch.linalg.norm(K_approx_2 - K_approx_1) / (K.shape[0]*K.shape[1])
            print(f"ML2 error for {kernel} kernel:", ML2_error.item())
            # print(kernel, K)
            
            assert approx_1_e < 0.1
            assert approx_2_e < 0.1
            assert ML2_error < 0.01
            #self.assertTrue(ML2_error < 0.01)

    ############################################################ 
    def Nystrom_K_shape_test(self):
        ############################################################ 

        # from pykeops.torch.nystrom import LazyNystrom_TK as Nystrom_TK
        ''' 
        Function to check that data shapes are correct.
    
        '''
        kernels = ['rbf', 'exp']
        
        for kernel in kernels:
            N_NT = Nystrom_TK(n_components=self.num_sampling, kernel = 'rbf', random_state=0).fit(self.x)

            assert N_NT.normalization_.shape == (self.num_sampling, self.num_sampling)
            assert N_NT.transform(self.x).shape == (self.length, self.num_sampling)
            #self.assertTrue(N_NT.normalization_.shape == (self.num_sampling, 1))
            #self.assertTrue(N_NT.transform(self.x).shape == (self.length, self.num_sampling))

print("float32 data")    
test_t = UnitTestCase(test_data_t, torch.float32)    
test_t.Nystrom_K_approx_test()
test_t.Nystrom_K_shape_test()

print("\n", "float64 data") 
test_t = UnitTestCase(test_data_t64, torch.float64)    
test_t.Nystrom_K_approx_test()
test_t.Nystrom_K_shape_test()

float32 data
rbf kernel K_approx 1 error 0.0007318102288991213
rbf kernel K_approx 2 error 0.0007318102288991213
ML2 error for rbf kernel: 2.0052039195750382e-10
exp kernel K_approx 1 error 0.0006974013522267342
exp kernel K_approx 2 error 0.0006974013522267342
ML2 error for exp kernel: 3.3612479466427203e-10

 float64 data
rbf kernel K_approx 1 error 0.0007318082706981316
rbf kernel K_approx 2 error 0.0007318082706981316
ML2 error for rbf kernel: 5.503805929069723e-19
exp kernel K_approx 1 error 0.0006974229535207323
exp kernel K_approx 2 error 0.0006974229535207322
ML2 error for exp kernel: 6.800603873377772e-19


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
